In [1]:
import torch
import numpy as np
from scipy import sparse
from gensim import corpora
from gensim import models
from gensim import utils
from torch_sparse import SparseTensor

In [2]:
doc=[]
for line in open("./data/doc.txt"):
    doc.append(line.replace("\n",""))
doc=[
    [word for word in line.split()] for line in doc
]
vocab=corpora.Dictionary(doc)
corpus=[vocab.doc2bow(text) for text in doc]

In [3]:
class MyCorpus:
    def __iter__(self,path="./data/"):
        for line in open(path+"doc.txt"):
            line=line.replace("\n","")
            line=[word for word in line.split()]
            yield line

In [4]:
sentences=MyCorpus()
model = models.Word2Vec(sentences=sentences,vector_size=300,min_count=1)

In [38]:
doc_vec=[]
for sentence in sentences:
    length=len(sentence)
    vector=[0]*300
    for word in sentence:
        vector+=model.wv[word]
    vector/=length
    doc_vec.append(vector)
doc_vec[0]

array([ 3.74677785e-02,  3.81630816e-01,  2.42912258e-02,  2.89468547e-01,
       -5.79276086e-02, -2.87689497e-01,  2.52352247e-01,  8.00950301e-01,
        9.91745594e-02, -1.77787834e-01,  1.59172626e-01, -2.46595068e-01,
       -1.85838942e-02,  2.90120893e-02, -2.39580540e-01, -2.37909989e-01,
        3.97150448e-01,  1.03251030e-03, -1.26464637e-02, -2.18395711e-01,
       -9.90426845e-02, -2.05516364e-02,  1.98573666e-01,  2.23734465e-01,
        1.10081054e-01, -2.48267207e-04, -3.74355820e-01,  1.18686790e-01,
       -2.78619063e-01, -3.74367914e-01,  1.04594191e-01, -1.41724523e-01,
        1.42250292e-01, -1.17747554e-01, -4.12916410e-02,  8.07079628e-02,
        1.40379630e-01, -4.01533716e-01, -3.04794056e-02,  3.07834383e-02,
       -2.02016060e-01,  1.05800099e-01,  3.48792850e-02, -2.82803718e-01,
        7.34995674e-02,  3.22645209e-01, -9.04785963e-03,  5.50526126e-02,
       -2.14875428e-02,  2.23067288e-01,  2.78744331e-02, -3.54776410e-02,
       -3.55410385e-01,  

In [31]:
vocab=[]
for line in open("./data/vocab.txt"):
    vocab.append(line.replace("\n",""))

In [28]:
word_vec=[]
for word in vocab:
    word_vec.append(model.wv[word])

In [29]:
doc_tensor=torch.Tensor(np.array(doc_vec))
word_tensor=torch.Tensor(np.array(word_vec))
data=torch.concat((doc_tensor,word_tensor),dim=0)

In [9]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    values=torch.FloatTensor(sparse_mx.data)
    indice=torch.LongTensor(np.vstack((sparse_mx.row,sparse_mx.col)).astype(np.int64))
    shape=torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indice,values,shape)

In [10]:
matrix=sparse.load_npz("./data/graph.npz")
matrix=sparse_mx_to_torch_sparse_tensor(matrix)

In [11]:
def load_features(path="./data/"):
    
    class MyCorpus:
        def __iter__(self):
            for line in open(path+"doc.txt"):
                line=line.replace("\n","")
                line=[word for word in line.split()]
                yield line
    sentences=MyCorpus()
    model = models.Word2Vec(sentences=sentences,vector_size=300,min_count=1)

    doc_vec=[]
    for idx,sentence in enumerate(sentences):
        length=len(sentence)
        for word in sentence:
            vector=[0]*300
            vector+=model.wv[word]
            vector/=length
        doc_vec.append(vector)
    
    vocab=[]
    for line in open(path+"vocab.txt"):
        vocab.append(line.replace("\n",""))
    word_vec=[]
    for word in vocab:
        word_vec.append(model.wv[word])

    doc_tensor=torch.Tensor(np.array(doc_vec))
    word_tensor=torch.Tensor(np.array(word_vec))
    features=torch.concat((doc_tensor,word_tensor),dim=0)

    return features

In [12]:
data=load_features()
data.shape

torch.Size([29426, 300])

In [13]:
def load_adj(path="./data/"):
    adj=sparse.load_npz("./data/graph.npz")
    return SparseTensor.from_scipy(adj)

adj=load_adj()
type(adj)

torch_sparse.tensor.SparseTensor

In [14]:
def load_label_idx(path="./data/"):
    raw=open(path+"train_split.txt")
    train_label_dict={}
    train_label=[]
    train_idx=[]
    test_label_dict={}
    test_label=[]
    test_idx=[]
    for line in raw:
        idx,status,label=line.split()
        idx,label=int(idx),int(label)
        if status=="train":
            train_idx.append(idx)
            train_label_dict[idx]=label
        else:
            test_idx.append(idx)
            test_label_dict[idx]=label

    for i in sorted(train_label_dict.keys()):
        train_label.append(train_label_dict[i])
    for i in sorted(test_label_dict.keys()):
        test_label.append(test_label_dict[i])

    train_label=torch.LongTensor(np.array(train_label))
    test_label=torch.LongTensor(np.array(test_label))
    labels=torch.concat((train_label,test_label),dim=0)

    train_idx=torch.LongTensor(np.array(train_idx))
    test_idx=torch.LongTensor(np.array(test_idx))

    return labels,train_idx,test_idx

In [15]:
labels,train_idx,test_idx=load_label_idx()

In [16]:
labels.shape,train_idx.shape,test_idx.shape

(torch.Size([10662]), torch.Size([7108]), torch.Size([3554]))

In [17]:
def load_data(path="./data/",device='cpu'):
    adj=load_adj(path).to(device)
    features=load_features(path).to(device)
    labels,train_idx,test_idx=load_label_idx(path)
    labels=labels.to(device=device)
    train_idx=train_idx.to(device=device)
    test_idx=test_idx.to(device=device)
    return adj,features,labels,train_idx,test_idx

In [18]:
adj, features, labels, idx_train, idx_test = load_data(device='cuda')

In [19]:
adj

SparseTensor(row=tensor([    0,     0,     0,  ..., 29425, 29425, 29425], device='cuda:0'),
             col=tensor([ 9619, 10001, 12466,  ..., 13838, 20571, 23614], device='cuda:0'),
             val=tensor([1.2822, 4.1746, 0.6764,  ..., 2.0778, 5.7191, 0.5558], device='cuda:0',
                          dtype=torch.float64),
             size=(29426, 29426), nnz=1898250, density=0.22%)